In [1]:
import os

# Check if we're in the root directory of rl-llm repo
if not os.path.basename(os.getcwd()) == 'rl-llm':
    # If we're in a subdirectory of rl-llm, find the root and cd to it
    current_path = os.getcwd()
    while os.path.basename(current_path) != 'rl-llm' and os.path.dirname(current_path) != current_path:
        current_path = os.path.dirname(current_path)
    
    if os.path.basename(current_path) == 'rl-llm':
        os.chdir(current_path)
        print(f"Changed directory to: {current_path}")
    else:
        print("Not in rl-llm repository structure")
else:
    print("Already in rl-llm root directory")

print(f"Current directory: {os.getcwd()}")

Changed directory to: /home/ubuntu/rl-llm
Current directory: /home/ubuntu/rl-llm


In [2]:
from src import EnvManager, sample_batch
import time
import matplotlib.pyplot as plt
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

/home/ubuntu/rl-llm/.venv/lib/python3.10/site-packages/gym/envs/registration.py:498: UserWarning: WARN: Overriding environment BabyAI-GoTo-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/home/ubuntu/rl-llm/.venv/lib/python3.10/site-packages/gym/envs/registration.py:498: UserWarning: WARN: Overriding environment BabyAI-Open-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/home/ubuntu/rl-llm/.venv/lib/python3.10/site-packages/gym/envs/registration.py:498: UserWarning: WARN: Overriding environment BabyAI-Pickup-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/home/ubuntu/rl-llm/.venv/lib/python3.10/site-packages/gym/envs/registration.py:498: UserWarning: WARN: Overriding environment BabyAI-PutNext-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
2025-04-03 16:53:14.317232: E external/

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_id = "meta-llama/Llama-3.2-3B-Instruct" # TODO: Test Llama-1B and Llama-3B (and optionally Qwen with reasoning)
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_id, padding_side="left")
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
generation_kwargs = {
    "max_new_tokens": 20,
    "do_sample": True,
    "top_k": 10,
    "top_p": 0.95,
    "temperature": 0.8,
    "pad_token_id": tokenizer.pad_token_id,
}
env_id = "BabyAI-PickUpSeqGoTo-v0" # TODO: Test GoTo, Pickup, PutNext, Open, and PickUpSeqGoTo
env_ids = [env_id]
context_window = 3 # TODO: Test 1, 2, 3, and 5 (if it's feasible)
num_envs = 4 # TODO: See what is optimal for you (might vary with model, env, and device)
envs = [
        EnvManager(
            env_ids, 
            invalid_action_penalty=-2,
            consecutive_invalid_actions_allowed=5,
        )
        for i in range(num_envs)
    ]
reasoning_flag = False
batch_size = 128
# TODO: You may also need to play around with the system_prompt to get valid actions if testing other models such as 1B or Qwen

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
total_times = []
total_generate_times = []
num_episodes_per_batch = []
successs = []
rewardss = []
episode_lengths = []
num_invalid_actions = []

num_batches = 5
for i in range(num_batches):

    start_time = time.time()
    queries, responses, rewards, stats, running_stats = sample_batch(
        envs,
        tokenizer,
        model,
        generation_kwargs,
        device,
        batch_size=batch_size,
        context_window=context_window,
        reasoning_flag = reasoning_flag,
    )
    end_time = time.time()
    sample_batch_time = end_time - start_time

    total_times.append(sample_batch_time)
    total_generate_times.append(stats["total_generate_time"])
    num_episodes = len(running_stats['success'][env_id])
    num_episodes_per_batch.append(num_episodes)

    successs.extend(running_stats['success'][env_id])
    rewardss.extend(running_stats['rewards'][env_id])
    episode_lengths.extend(running_stats['episode_lengths'][env_id])
    num_invalid_actions.extend(running_stats['num_invalid_actions'][env_id])
    print(f"Batch {i+1}/{num_batches}:")
    print(f"  Sample batch time: {sample_batch_time:.2f} seconds")
    print(f"  Total generate time: {stats['total_generate_time']:.2f} seconds")
    print(f"  Number of episodes: {num_episodes}")

Batch 1/5:
  Sample batch time: 35.79 seconds
  Total generate time: 35.58 seconds
  Number of episodes: 1
Batch 2/5:
  Sample batch time: 35.06 seconds
  Total generate time: 34.82 seconds
  Number of episodes: 4
Batch 3/5:
  Sample batch time: 37.00 seconds
  Total generate time: 36.77 seconds
  Number of episodes: 3
Batch 4/5:
  Sample batch time: 37.05 seconds
  Total generate time: 36.82 seconds
  Number of episodes: 4
Batch 5/5:
  Sample batch time: 35.63 seconds
  Total generate time: 35.40 seconds
  Number of episodes: 4


In [5]:
print("Experiment:")
print(f"Model: {model_id}")
print(f"Environment: {env_id}")
print(f"Context Window: {context_window}")
print(f"Number of environments: {num_envs}")
print(f"Reasoning: {'Enabled' if reasoning_flag else 'Disabled'}")
print()

print("Summary of results:")
print(f"Total batches processed: {num_batches}")
print(f"Average sample batch time: {sum(total_times)/num_batches:.2f} seconds")
print(f"Average total generate time: {sum(total_generate_times)/num_batches:.2f} seconds")
print(f"Number of episodes per batch ranges between: {min(num_episodes_per_batch)} to {max(num_episodes_per_batch)}")
print(f"Total number of episodes: {sum(num_episodes_per_batch)}")
print(f"Success rate: {sum(successs) / len(successs) * 100:.2f}%")
print(f"Average rewards: {sum(rewardss) / len(rewardss):.2f}")
print(f"Average episode length: {sum(episode_lengths) / len(episode_lengths):.2f}")
print(f"Average number of invalid actions: {sum(num_invalid_actions) / len(num_invalid_actions):.2f}")

Experiment:
Model: meta-llama/Llama-3.2-3B-Instruct
Environment: BabyAI-PickUpSeqGoTo-v0
Context Window: 3
Number of environments: 4
Reasoning: Disabled

Summary of results:
Total batches processed: 5
Average sample batch time: 36.11 seconds
Average total generate time: 35.88 seconds
Number of episodes per batch ranges between: 1 to 4
Total number of episodes: 16
Success rate: 6.25%
Average rewards: 0.06
Average episode length: 120.62
Average number of invalid actions: 0.00


## Results

NOTE: GoTo works well even with context widnow of 1
```Experiment:
Model: meta-llama/Llama-3.2-3B-Instruct
Environment: BabyAI-GoTo-v0
Context Window: 1
Number of environments: 4
Reasoning: Disabled

Summary of results:
Total batches processed: 10
Average sample batch time: 9.77 seconds
Average total generate time: 9.65 seconds
Number of episodes per batch ranges between: 4 to 8
Success rate: 74.58%
Average rewards: 0.52
Average episode length: 32.49
Average number of invalid actions: 0.00

NOTE: Pickup has no success with the same settings as prev GoTo experiment
```Experiment:
Model: meta-llama/Llama-3.2-3B-Instruct
Environment: BabyAI-Pickup-v0
Context Window: 1
Number of environments: 4
Reasoning: Disabled

Summary of results:
Total batches processed: 10
Average sample batch time: 10.08 seconds
Average total generate time: 9.96 seconds
Number of episodes per batch ranges between: 4 to 4
Success rate: 0.00%
Average rewards: 0.00
Average episode length: 64.00
Average number of invalid actions: 0.00

NOTE: Changed prompt but did not seem to help
```Experiment:
Model: meta-llama/Llama-3.2-3B-Instruct
Environment: BabyAI-Pickup-v0
Context Window: 1
Number of environments: 4
Reasoning: Disabled

Summary of results:
Total batches processed: 10
Average sample batch time: 14.32 seconds
Average total generate time: 14.21 seconds
Number of episodes per batch ranges between: 2 to 4
Success rate: 0.00%
Average rewards: 0.00
Average episode length: 64.00
Average number of invalid actions: 0.00

NOTE: Success! Some sign of life with increased context window.
```Experiment:
Model: meta-llama/Llama-3.2-3B-Instruct
Environment: BabyAI-Pickup-v0
Context Window: 3
Number of environments: 4
Reasoning: Disabled

Summary of results:
Total batches processed: 5
Average sample batch time: 20.86 seconds
Average total generate time: 20.75 seconds
Number of episodes per batch ranges between: 2 to 4
Success rate: 13.33%
Average rewards: 0.11
Average episode length: 57.13
Average number of invalid actions: 1.07

NOTE: The old prompt works even better!
```Experiment:
Model: meta-llama/Llama-3.2-3B-Instruct
Environment: BabyAI-Pickup-v0
Context Window: 3
Number of environments: 4
Reasoning: Disabled

Summary of results:
Total batches processed: 5
Average sample batch time: 17.13 seconds
Average total generate time: 17.01 seconds
Number of episodes per batch ranges between: 3 to 4
Success rate: 42.11%
Average rewards: 0.30
Average episode length: 45.42
Average number of invalid actions: 0.05

NOTE: Further increasing context window to 5 yielded slightly better results.
```Experiment:
Model: meta-llama/Llama-3.2-3B-Instruct
Environment: BabyAI-Pickup-v0
Context Window: 5
Number of environments: 4
Reasoning: Disabled

Summary of results:
Total batches processed: 5
Average sample batch time: 23.26 seconds
Average total generate time: 23.13 seconds
Number of episodes per batch ranges between: 3 to 6
Total number of episodes: 22
Success rate: 45.45%
Average rewards: 0.31
Average episode length: 41.55
Average number of invalid actions: 0.73

NOTE: Sanity check with context window back to 3 see that results are reasonably reporoducible.
```Experiment:
Model: meta-llama/Llama-3.2-3B-Instruct
Environment: BabyAI-Pickup-v0
Context Window: 3
Number of environments: 4
Reasoning: Disabled

Summary of results:
Total batches processed: 5
Average sample batch time: 17.33 seconds
Average total generate time: 17.21 seconds
Number of episodes per batch ranges between: 3 to 7
Total number of episodes: 22
Success rate: 40.91%
Average rewards: 0.31
Average episode length: 44.68
Average number of invalid actions: 0.09

NOTE: Even context size of 2 seems to be sufficient for Pickup (with much faster sample batch time)
```Experiment:
Model: meta-llama/Llama-3.2-3B-Instruct
Environment: BabyAI-Pickup-v0
Context Window: 2
Number of environments: 4
Reasoning: Disabled

Summary of results:
Total batches processed: 5
Average sample batch time: 13.91 seconds
Average total generate time: 13.79 seconds
Number of episodes per batch ranges between: 2 to 5
Total number of episodes: 18
Success rate: 38.89%
Average rewards: 0.20
Average episode length: 52.67
Average number of invalid actions: 0.11

NOTE: Just confirming that context size 1 defineitely does not work for Pickup
```Experiment:
Model: meta-llama/Llama-3.2-3B-Instruct
Environment: BabyAI-Pickup-v0
Context Window: 1
Number of environments: 4
Reasoning: Disabled

Summary of results:
Total batches processed: 5
Average sample batch time: 9.92 seconds
Average total generate time: 9.80 seconds
Number of episodes per batch ranges between: 4 to 4
Total number of episodes: 20
Success rate: 0.00%
Average rewards: 0.00
Average episode length: 64.00
Average number of invalid actions: 0.00

NOTE: PutNext also does not work for context window of 1
```Experiment:
Model: meta-llama/Llama-3.2-3B-Instruct
Environment: BabyAI-PutNext-v0
Context Window: 1
Number of environments: 4
Reasoning: Disabled

Summary of results:
Total batches processed: 5
Average sample batch time: 21.00 seconds
Average total generate time: 20.76 seconds
Number of episodes per batch ranges between: 3 to 4
Total number of episodes: 19
Success rate: 0.00%
Average rewards: 0.00
Average episode length: 128.00
Average number of invalid actions: 0.00

NOTE: Even context window of 2 does not work for PutNext
```Experiment:
Model: meta-llama/Llama-3.2-3B-Instruct
Environment: BabyAI-PutNext-v0
Context Window: 2
Number of environments: 4
Reasoning: Disabled

Summary of results:
Total batches processed: 5
Average sample batch time: 28.63 seconds
Average total generate time: 28.39 seconds
Number of episodes per batch ranges between: 1 to 3
Total number of episodes: 11
Success rate: 0.00%
Average rewards: 0.00
Average episode length: 128.00
Average number of invalid actions: 0.00

NOTE: Decent success with context window of 5 on PutNext (at the expense of longer sample batch time)
```Experiment:
Model: meta-llama/Llama-3.2-3B-Instruct
Environment: BabyAI-PutNext-v0
Context Window: 5
Number of environments: 4
Reasoning: Disabled

Summary of results:
Total batches processed: 5
Average sample batch time: 48.36 seconds
Average total generate time: 48.13 seconds
Number of episodes per batch ranges between: 1 to 3
Total number of episodes: 11
Success rate: 18.18%
Average rewards: 0.08
Average episode length: 119.09
Average number of invalid actions: 0.18

NOTE: Small success with context window of 3 on PutNext, but not as good as 5
```Experiment:
Model: meta-llama/Llama-3.2-3B-Instruct
Environment: BabyAI-PutNext-v0
Context Window: 3
Number of environments: 4
Reasoning: Disabled

Summary of results:
Total batches processed: 5
Average sample batch time: 35.11 seconds
Average total generate time: 34.88 seconds
Number of episodes per batch ranges between: 1 to 4
Total number of episodes: 13
Success rate: 7.69%
Average rewards: 0.04
Average episode length: 123.85
Average number of invalid actions: 0.00

NOTE: Even Open works with context window of 5!
```Experiment:
Model: meta-llama/Llama-3.2-3B-Instruct
Environment: BabyAI-Open-v0
Context Window: 5
Number of environments: 4
Reasoning: Disabled

Summary of results:
Total batches processed: 5
Average sample batch time: 39.76 seconds
Average total generate time: 39.57 seconds
Number of episodes per batch ranges between: 2 to 2
Total number of episodes: 10
Success rate: 10.00%
Average rewards: 0.06
Average episode length: 88.20
Average number of invalid actions: 3.40

NOTE: Prev 10% success rate might have been a fluke, since there was no success in 3 more batches
```Experiment:
Model: meta-llama/Llama-3.2-3B-Instruct
Environment: BabyAI-Open-v0
Context Window: 5
Number of environments: 4
Reasoning: Disabled

Summary of results:
Total batches processed: 3
Average sample batch time: 37.97 seconds
Average total generate time: 37.78 seconds
Number of episodes per batch ranges between: 2 to 4
Total number of episodes: 9
Success rate: 0.00%
Average rewards: 0.00
Average episode length: 82.11
Average number of invalid actions: 3.33

NOTE: Maybe not, since there is a non-zero success rate even with context window of 3
```Experiment:
Model: meta-llama/Llama-3.2-3B-Instruct
Environment: BabyAI-Open-v0
Context Window: 3
Number of environments: 4
Reasoning: Disabled

Summary of results:
Total batches processed: 5
Average sample batch time: 35.46 seconds
Average total generate time: 35.23 seconds
Number of episodes per batch ranges between: 2 to 3
Total number of episodes: 12
Success rate: 8.33%
Average rewards: 0.04
Average episode length: 123.25
Average number of invalid actions: 0.17

NOTE: Context window of 5 is sufficient for PickUpSeqGoTo too!
```Experiment:
Model: meta-llama/Llama-3.2-3B-Instruct
Environment: BabyAI-PickUpSeqGoTo-v0
Context Window: 5
Number of environments: 4
Reasoning: Disabled

Summary of results:
Total batches processed: 5
Average sample batch time: 47.41 seconds
Average total generate time: 47.18 seconds
Number of episodes per batch ranges between: 2 to 4
Total number of episodes: 13
Success rate: 38.46%
Average rewards: 0.18
Average episode length: 108.23
Average number of invalid actions: 0.15

NOTE: Small success rate with context window of 3, but not as good as 5 for PickUpSeqGoTo
```Experiment:
Model: meta-llama/Llama-3.2-3B-Instruct
Environment: BabyAI-PickUpSeqGoTo-v0
Context Window: 3
Number of environments: 4
Reasoning: Disabled

Summary of results:
Total batches processed: 5
Average sample batch time: 36.11 seconds
Average total generate time: 35.88 seconds
Number of episodes per batch ranges between: 1 to 4
Total number of episodes: 16
Success rate: 6.25%
Average rewards: 0.06
Average episode length: 120.62
Average number of invalid actions: 0.00

TODO:
- [ ] Compare with other models
- [ ] Compare reasoning with non-reasoning
- [ ] Vary the number of distractors
- [ ] Figure out the optimal setting of num_envs for time efficiency

NOTE: We can try GoTo, and PickUpSeqGoTo to get a sense for these other experiments?